In [ ]:
##NEW
import numpy as np
import pandas as pd
import glob
import os 

ID = 'nvasofg2'
vector = np.arange(1, 169)  # 168 TRs total
matrix = vector.reshape(12, 14) - 1  # 12 routines, each with 14 TRs
mat_df = pd.DataFrame(matrix)

#csv_path = f'/System/Volumes/Data/misc/data17/persichettias/vasoFG/{ID}/data/100_famousVaso_2025-01-08_14h06.33.557.csv'
subdirs = glob.glob(f'/System/Volumes/Data/misc/data17/persichettias/vasoFG/{ID}/data/100_famousVaso*')

#df_csv = pd.read_csv(csv_path) 

for dir in subdirs: 
    df_csv = pd.read_csv(dir)
    filename = os.path.basename(dir).split('.')[2] #run number 1-6
    #print(filename)
    
#map to change name (perceive/imagine) 
    routine_name_map = {
    "face_name_reps": "imagineFace",
    "face_image_reps": "perceiveFace",
    "place_name_reps": "imaginePlace",
    "place_image_reps": "perceivePlace"
}
    
# dictionary for routine TRs
    routine_trs = {
        "face_name_reps": [],
        "face_image_reps": [],
        "place_name_reps": [],
        "place_image_reps": []
}
    
    #routine_sequence = []
    
    # Rows where routines appear (= 1)
    rows = [2, 20, 38, 56, 74, 92, 110, 128, 146, 164, 182, 200]  
    cols = list(routine_trs.keys())
    
    for col in cols:   
        routine_trs[col] = []
    #GRAB MATRIX ROW BASED ON LOOP INDEX
        for i, x in enumerate(rows):  # Iterate over rows , saves index of loop (i)
            if df_csv[col].iloc[x] == 1:     # If routine was shown at this row.. 
                if i < len(mat_df):    #error check for 0-based index
                    tr_values = mat_df.iloc[i].values      #Extract the correct 14 TRs using integer quotient
                    routine_trs[col].extend(tr_values)   # adds TRs to dict one by one
                else:
                    print(f"Warning: Iteration {i} skipped  -  out of bounds.")
                    
    # #prints routine order  
    #     routine_sequence.append(col)
    # df_routine_sequence = pd.DataFrame({'Routine': routine_sequence})   
    # # Prints routine order
    # print("Routine Sequence:")
    # print(df_routine_sequence)
        
#Conv routine_trs dict into a DataFrame
    df_routines = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in routine_trs.items()]))
    #print(df_routines)
    
    output_path = '/Users/baffoebonnieja/Downloads/'
    
    six_TRs = []
    for routine, TR_matrix in routine_trs.items(): #dataFrame version (key/ val)
        new_routine_name = routine_name_map.get(routine, routine)  # Get new name
        TR_matrix = np.array(TR_matrix).reshape(-1, 14) ##reshape: -1: determines num of rows, ensures each is 14
        eight_TRs = [str(tr) for row in TR_matrix for tr in row[:8]]
        
        for row in TR_matrix: 
            six_TRs.extend(row[-6:])                
        six_TRs.sort()
    #VOLS
        TR_path = f'{output_path}{ID}_{new_routine_name}_vols_{filename}.txt'
        with open(TR_path, 'w') as f:
            f.write(','.join(eight_TRs))
        print(f'Saved: {ID}_{new_routine_name}_vols_{filename}.txt')

        
    #FIXATION , outside of loop (only 1 file) 
    fix_path = f'{output_path}{ID}_fixation_{filename}.txt'
    with open(fix_path, 'w') as f:
        f.write(','.join(map(str, six_TRs)))
    print(f'Saved: {ID}_fixation_{filename}.txt')


In [ ]:
#LOCALIZER
ID = 'nvasofg1'

import numpy as np 
import pandas as pd

vector = np.arange(1, 169)
matrix = vector.reshape(12, 14)
#matrix
new_mat = matrix - 1
#print(new_mat)

df = pd.DataFrame(new_mat)

output_path = '/System/Volumes/Data/misc/data17/persichettias/vasoFG/localizer/'

#assigns alternating blocks for LOC   
PlaceImage = [df.iloc[[1]], df.iloc[[3]], df.iloc[[5]], df.iloc[[7]], df.iloc[[9]], df.iloc[[11]]]
FaceImage = [df.iloc[[0]], df.iloc[[2]], df.iloc[[4]], df.iloc[[6]], df.iloc[[8]], df.iloc[[10]]]

PlaceImage_concat = pd.concat(PlaceImage, ignore_index=True)
FaceImage_concat = pd.concat(FaceImage, ignore_index=True)

loc_conditions = {'place': PlaceImage_concat,'face': FaceImage_concat}                

#FIXATION
for condition_name, value in loc_conditions.items():   
    fix_path = f'{output_path}fixation_vols.txt'
    with open(fix_path, 'w') as f:
        f.write(','.join(map(str, df.iloc[:, -6:].values.flatten())))  #flattens last 3 col to one line as a comma sep string
print(f'Saved: {condition_name}_LOC_fixation')

#VOLS
for condition_name, value in loc_conditions.items():   
    TR_path = f'{output_path}{condition_name}_vols.txt'
    row_str = ','.join(map(str, value.iloc[:, :8].values.flatten()))
    with open(TR_path, 'w') as f:
        f.write(row_str)
print(f'Saved: {condition_name}_LOC_vols')



In [15]:
##OLD

#14 TRs per block(per row) | outputs specific TRs or timing of each block
#vols: first 4 
#TRs: last 3 
ID = 'nvasofg2'

import numpy as np 
import pandas as pd

vector = np.arange(1, 169)
matrix = vector.reshape(12, 14)
#matrix
new_mat = matrix - 1
new_mat

df = pd.DataFrame(new_mat)

#make broad path
output_path = f'/System/Volumes/Data/misc/data17/persichettias/vasoFG/{ID}/timing/'

#perceiveFace imagineFace perceivePlace imaginePlace

## assigns rows/blocks to a condition based on psychopy routine / change #s each subj (0-11)
# PlaceName = [df.iloc[[1]], df.iloc[[7]], df.iloc[[11]]]   #subj1
# FaceImage = [df.iloc[[0]], df.iloc[[5]], df.iloc[[9]]]
# PlaceImage = [df.iloc[[2]], df.iloc[[6]], df.iloc[[8]]]
# FaceName = [df.iloc[[3]], df.iloc[[4]], df.iloc[[10]]]

imaginePlace = [df.iloc[[0]], df.iloc[[6]], df.iloc[[8]]]      ##subj2
perceiveFace = [df.iloc[[1]], df.iloc[[5]], df.iloc[[10]]]
perceivePlace = [df.iloc[[2]], df.iloc[[4]], df.iloc[[9]]]
imagineFace = [df.iloc[[3]], df.iloc[[7]], df.iloc[[11]]]

imaginePlace_concat = pd.concat(imaginePlace, ignore_index=True)
perceivePlace_concat = pd.concat(perceivePlace, ignore_index=True)
perceiveFace_concat = pd.concat(perceiveFace, ignore_index=True)
imagineFace_concat = pd.concat(imagineFace, ignore_index=True)

#str format / one line
# FaceName_str = ''.join(FaceName_concat.astype(str).values.flatten())
# print(FaceName_str)

conditions = {'imaginePlace': imagineFace_concat,'perceivePlace': perceivePlace_concat,'perceiveFace': 
              perceiveFace_concat,'imaginePlace':imaginePlace_concat}

# FIXATION
for condition_name, value in conditions.items():   #key, value notation 
    fix_path = f'{output_path}{ID}_fixation.txt'
    with open(fix_path, 'w') as f:
        f.write(','.join(map(str, df.iloc[:, -6:].values.flatten())))  #flattens last 3 col to one line as a comma sep string
print(f'Saved: {ID}_{condition_name}_fix')

#VOLS
for condition_name, value in conditions.items():   #key, value notation
    TR_path = f'{output_path}{ID}_{condition_name}_vols.txt'
    row_str = ','.join(map(str, value.iloc[:, :8].values.flatten()))
    with open(TR_path, 'w') as f:
        f.write(row_str)
    print(f'Saved: {ID}_{condition_name}_vols')
    

Saved: nvasofg2_PlaceName_fix
Saved: nvasofg2_FaceName_vols
Saved: nvasofg2_PlaceImage_vols
Saved: nvasofg2_FaceImage_vols
Saved: nvasofg2_PlaceName_vols


In [6]:
print(new_mat)

[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13]
 [ 14  15  16  17  18  19  20  21  22  23  24  25  26  27]
 [ 28  29  30  31  32  33  34  35  36  37  38  39  40  41]
 [ 42  43  44  45  46  47  48  49  50  51  52  53  54  55]
 [ 56  57  58  59  60  61  62  63  64  65  66  67  68  69]
 [ 70  71  72  73  74  75  76  77  78  79  80  81  82  83]
 [ 84  85  86  87  88  89  90  91  92  93  94  95  96  97]
 [ 98  99 100 101 102 103 104 105 106 107 108 109 110 111]
 [112 113 114 115 116 117 118 119 120 121 122 123 124 125]
 [126 127 128 129 130 131 132 133 134 135 136 137 138 139]
 [140 141 142 143 144 145 146 147 148 149 150 151 152 153]
 [154 155 156 157 158 159 160 161 162 163 164 165 166 167]]


In [16]:
#LOCALIZER
ID = 'nvasofg1'

import numpy as np 
import pandas as pd

vector = np.arange(1, 169)
matrix = vector.reshape(12, 14)
#matrix
new_mat = matrix - 1
#print(new_mat)

df = pd.DataFrame(new_mat)

output_path = '/System/Volumes/Data/misc/data17/persichettias/vasoFG/localizer/'

#assigns alternating blocks for LOC   
PlaceImage = [df.iloc[[1]], df.iloc[[3]], df.iloc[[5]], df.iloc[[7]], df.iloc[[9]], df.iloc[[11]]]
FaceImage = [df.iloc[[0]], df.iloc[[2]], df.iloc[[4]], df.iloc[[6]], df.iloc[[8]], df.iloc[[10]]]

PlaceImage_concat = pd.concat(PlaceImage, ignore_index=True)
FaceImage_concat = pd.concat(FaceImage, ignore_index=True)

loc_conditions = {'place': PlaceImage_concat,'face': FaceImage_concat}                

#FIXATION
for condition_name, value in loc_conditions.items():   
    fix_path = f'{output_path}fixation_vols.txt'
    with open(fix_path, 'w') as f:
        f.write(','.join(map(str, df.iloc[:, -6:].values.flatten())))  #flattens last 3 col to one line as a comma sep string
print(f'Saved: {condition_name}_LOC_fixation')

#VOLS
for condition_name, value in loc_conditions.items():   
    TR_path = f'{output_path}{condition_name}_vols.txt'
    row_str = ','.join(map(str, value.iloc[:, :8].values.flatten()))
    with open(TR_path, 'w') as f:
        f.write(row_str)
print(f'Saved: {condition_name}_LOC_vols')



Saved: nvasofg1_face_LOC_fixation
Saved: nvasofg1_face_LOC_vols


In [71]:
##OLD

import numpy as np
import pandas as pd

# Subject ID
ID = 'nvasofg2'

vector = np.arange(1, 169)
matrix = vector.reshape(12, 14) - 1  # start at zero
mat_df = pd.DataFrame(matrix)

csv_path = f'/System/Volumes/Data/misc/data17/persichettias/vasoFG/{ID}/data/100_famousVaso_2025-01-08_14h06.33.557.csv'
df_csv = pd.read_csv(csv_path)  # Load the CSV with randomized order
#LIST
# routine_groups = [
# 'place_image_reps' == [],
# 'face_image_reps' == [],
# 'place_name_reps' == [],
# 'face_name_reps' == [] 
# ]

#DICT
routine_groups = {
'place_image_reps' : [],
'face_image_reps' : [],
'place_name_reps' : [],
'face_name_reps' : [] 
}


#rows = [2, 20, 38]
cols = ['place_image_reps','face_image_reps','place_name_reps','face_name_reps']

for col in cols:
    #for x in rows:
        if df_csv[col].iloc[2] == 1: 
            routine_groups[col].append(mat_df.iloc[0])  # add the first row of mat_df if theres a 1 from 2-19
            
        elif df_csv[col].iloc[20] == 1: 
            routine_groups[col].append(mat_df.iloc[1])

        elif df_csv[col].iloc[38] == 1: 
            routine_groups[col].append(mat_df.iloc[2])

        elif df_csv[col].iloc[56] == 1: 
            routine_groups[col].append(mat_df.iloc[3])

df_routines = pd.DataFrame(routine_groups)
print(df_routines)
            
          #appends 

            
# FaceImage = [df.iloc[[0]], df.iloc[[2]], df.iloc[[4]], df.iloc[[6]], df.iloc[[8]], df.iloc[[10]]]
# PlaceImage_concat = pd.concat(PlaceImage, ignore_index=True)
        
        
#         row = df_csv[].iloc[x] #replace w conditions
#         if row_index == 1:    #if value at rows is 1, save column name
#             print(df_csv.columns(.iloc[x]))
#     # for x in condition:
#     match1_index = df_csv['place_image_reps'].eq(1).idxmax()
# print('match', match1_index)

# n = 2  # Change this for the N-th occurrence
# indices = df[df['A'] == 1].index

# if len(indices) >= n:
#     nth_index = indices[n - 1]  # Zero-based indexing
#     print(nth_index)
# else:
#     print("Not enough occurrences")

        
        


# select row(0) 
# block_order = df_csv['objectstim'].tolist()  # Extract sequence of routines
# Unique routine categories
# unique_routines = set(block_order)

# # Dictionary to store matrix rows for each routine
# routine_mapping = {routine: [] for routine in unique_routines}

# for idx, routine in enumerate(block_order):
#     routine_mapping[routine].append(df.iloc[idx])  # Assign row dynamically

# # Convert lists to DataFrames
# for key in routine_mapping:
#     routine_mapping[key] = pd.concat(routine_mapping[key], ignore_index=True)

# # Define output path
# output_path = f'/System/Volumes/Data/misc/data17/persichettias/vasoFG/{ID}/timing/'

# # Save dynamically assigned rows to text files
# for routine, data in routine_mapping.items():
#     file_path = f'{output_path}{routine}.txt'
#     data.to_csv(file_path, sep=',', header=False, index=False)
#     print(f'Saved: {file_path}')


                                    place_image_reps  \
0  0     28
1     29
2     30
3     31
4     32
5...   

                                     face_image_reps  \
0  0     14
1     15
2     16
3     17
4     18
5...   

                                     place_name_reps  \
0  0      0
1      1
2      2
3      3
4      4
5...   

                                      face_name_reps  
0  0     42
1     43
2     44
3     45
4     46
5...  


In [54]:
mat_df.iloc[0,0]

0

In [53]:
mat_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13
1,14,15,16,17,18,19,20,21,22,23,24,25,26,27
2,28,29,30,31,32,33,34,35,36,37,38,39,40,41
3,42,43,44,45,46,47,48,49,50,51,52,53,54,55
4,56,57,58,59,60,61,62,63,64,65,66,67,68,69
5,70,71,72,73,74,75,76,77,78,79,80,81,82,83
6,84,85,86,87,88,89,90,91,92,93,94,95,96,97
7,98,99,100,101,102,103,104,105,106,107,108,109,110,111
8,112,113,114,115,116,117,118,119,120,121,122,123,124,125
9,126,127,128,129,130,131,132,133,134,135,136,137,138,139


Saved: nvasofg2_imagineFace_vols_6.txt
Saved: nvasofg2_perceiveFace_vols_6.txt
Saved: nvasofg2_imaginePlace_vols_6.txt
Saved: nvasofg2_perceivePlace_vols_6.txt
Saved: nvasofg2_imagineFace_vols_2.txt
Saved: nvasofg2_perceiveFace_vols_2.txt
Saved: nvasofg2_imaginePlace_vols_2.txt
Saved: nvasofg2_perceivePlace_vols_2.txt
Saved: nvasofg2_imagineFace_vols_3.txt
Saved: nvasofg2_perceiveFace_vols_3.txt
Saved: nvasofg2_imaginePlace_vols_3.txt
Saved: nvasofg2_perceivePlace_vols_3.txt
Saved: nvasofg2_imagineFace_vols_1.txt
Saved: nvasofg2_perceiveFace_vols_1.txt
Saved: nvasofg2_imaginePlace_vols_1.txt
Saved: nvasofg2_perceivePlace_vols_1.txt
Saved: nvasofg2_imagineFace_vols_5.txt
Saved: nvasofg2_perceiveFace_vols_5.txt
Saved: nvasofg2_imaginePlace_vols_5.txt
Saved: nvasofg2_perceivePlace_vols_5.txt
Saved: nvasofg2_imagineFace_vols_4.txt
Saved: nvasofg2_perceiveFace_vols_4.txt
Saved: nvasofg2_imaginePlace_vols_4.txt
Saved: nvasofg2_perceivePlace_vols_4.txt


In [53]:
routine_trs

['8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '148',
 '149',
 '150',
 '151',
 '152',
 '153',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139']

In [86]:
df_routines

,Routines
place_image_reps,"[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 3..."
face_image_reps,"[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2..."
place_name_reps,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
face_name_reps,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5..."


In [116]:
tr_values

array([154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167])

In [120]:
pd.set_option('display.max_rows', None)
print(df_routines)
#pd.reset_option('display.max_rows')  # Reset back to default afte

                                                           Routines
place_image_reps  [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 3...
face_image_reps   [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2...
place_name_reps   [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
face_name_reps    [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5...


In [122]:
import numpy as np
import pandas as pd

ID = 'nvasofg2'
output_path = '/Users/baffoebonnieja/Downloads/'

# Generate 12x14 matrix
vector = np.arange(1, 169)  # 168 TRs total
matrix = vector.reshape(12, 14) - 1  # 12 routines, each with 14 TRs
mat_df = pd.DataFrame(matrix)

# Read CSV
csv_path = '/Users/baffoebonnieja/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/vasoFamousFP/data/100_famousVaso_2025-01-08_14h06.33.557.csv'
df_csv = pd.read_csv(csv_path)

# Dict to store routine TRs
routine_trs = {
    'place_image_reps': [],
    'face_image_reps': [],
    'place_name_reps': [],
    'face_name_reps': []
}

# Rows where routines appear (= 1)
rows = [2, 20, 38, 56, 74, 92, 110, 128, 146, 164, 182, 200]  
cols = list(routine_trs.keys())

for col in cols:
    for i, x in enumerate(rows):  # Iterate over row indices
        if df_csv[col].iloc[x] == 1:  # If routine was shown at this row  
            if i < len(mat_df):  # Ensure index is within bounds
                tr_values = mat_df.iloc[i].values  # Extract 14 TRs for this routine
                routine_trs[col].append(tr_values)  # Append as a list

# Convert to NumPy array for cleaner manipulation
for routine, trs_list in routine_trs.items():
    routine_trs[routine] = np.array(trs_list)  # Convert each to NumPy array

# Write each routine's unique values to separate text files
for routine, trs_matrix in routine_trs.items():
    # Extract first 8 TRs from each row correctly
    first_8_TRs = [','.join(map(str, row[:8])) for row in trs_matrix]

    # Save as a text file
    TR_path = f'{output_path}{routine}_vols.txt'
    with open(TR_path, 'w') as f:
        f.write('\n'.join(first_8_TRs))  # Write each row on a new line
    print(f'Saved: {TR_path}')

    # FIXATION (last 6 TRs)
    fixation_TRs = [','.join(map(str, row[-6:])) for row in trs_matrix]
    
    fix_path = f'{output_path}{ID}_{routine}_fixation.txt'
    with open(fix_path, 'w') as f:
        f.write('\n'.join(fixation_TRs))
    print(f'Saved: {fix_path}')


Saved: /Users/baffoebonnieja/Downloads/place_image_reps_vols.txt
Saved: /Users/baffoebonnieja/Downloads/nvasofg2_place_image_reps_fixation.txt
Saved: /Users/baffoebonnieja/Downloads/face_image_reps_vols.txt
Saved: /Users/baffoebonnieja/Downloads/nvasofg2_face_image_reps_fixation.txt
Saved: /Users/baffoebonnieja/Downloads/place_name_reps_vols.txt
Saved: /Users/baffoebonnieja/Downloads/nvasofg2_place_name_reps_fixation.txt
Saved: /Users/baffoebonnieja/Downloads/face_name_reps_vols.txt
Saved: /Users/baffoebonnieja/Downloads/nvasofg2_face_name_reps_fixation.txt


In [1]:
subdirs

NameError: name 'subdirs' is not defined